In [ ]:
# TCR & BCR
df = adata.obs[["AIRR_Type_BCR","AIRR_Type_TCR"]]
df["AIRR_Type_BCR"] = pd.Categorical(df["AIRR_Type_BCR"], categories=['TCR', 'BCR'], ordered=True)
df["AIRR_Type_TCR"] = pd.Categorical(df["AIRR_Type_TCR"], categories=['TCR', 'BCR'], ordered=True)
df["AIRR_Type"] = df["AIRR_Type_BCR"]
df["AIRR_Type"] = df["AIRR_Type"].fillna(df["AIRR_Type_TCR"])
adata.obs["AIRR_Type"] = df["AIRR_Type"]

In [ ]:
# TCR Clone Size per Sample
adata.obs["raw_clonotype_id_TCR"] = adata.obs["raw_clonotype_id_TCR"].str.replace("sample0_","")
adata.obs["Sample"] = adata.obs["Sample"].astype(str)
adata.obs["raw_clonotype_id_TCR"] = adata.obs["Sample"] + "_" + adata.obs["raw_clonotype_id_TCR"]
df = adata.obs[["Sample", "raw_clonotype_id_TCR"]].value_counts()
df = df.reset_index()
df.columns = ["Sample","raw_clonotype_id_TCR","TCR_CloneSize_perSample"]
df_obs = adata.obs
df_obs = df_obs.merge(df, on="raw_clonotype_id_TCR", how="left")
print(list(adata.obs.index)==list(df_obs.BC))
adata.obs["TCR_CloneSize_perSample"] = list(df_obs.TCR_CloneSize_perSample)

In [ ]:
# Identify BCR clone
df_bcr = pd.read_csv("/path/UCART_SLE_6Samples_BCRClone.csv")
df_bcr["barcode"] = df_bcr["barcode"].str.split("_").str[1]
df_bcr["barcode"] = df_bcr["barcode"] + "-" + df_bcr["sample"]
df_bcr = df_bcr.add_prefix('BCR_')
df_obs = adata.obs
df = pd.merge(adata.obs, df_bcr, left_on = "BC", right_on="BCR_barcode", how="left")
print(list(adata.obs.index)==list(df["BC"]))
adata.obs["BCR_IGH"] = list(df["BCR_IGH"])
adata.obs["BCR_cdr3_aa1"] = list(df["BCR_cdr3_aa1"])
adata.obs["BCR_cdr3_nt1"] = list(df["BCR_cdr3_nt1"])
adata.obs["BCR_IGLC"] = list(df["BCR_IGLC"])
adata.obs["BCR_cdr3_aa2"] = list(df["BCR_cdr3_aa2"])
adata.obs["BCR_cdr3_nt2"] = list(df["BCR_cdr3_nt2"])
adata.obs["BCR_CTgene"] = list(df["BCR_CTgene"])
adata.obs["BCR_CTnt"] = list(df["BCR_CTnt"])
adata.obs["BCR_CTaa"] = list(df["BCR_CTaa"])
adata.obs["BCR_CTstrict"] = list(df["BCR_CTstrict"])
adata.obs["BCR_cellType"] = list(df["BCR_cellType"])

df = adata.obs["BCR_CTstrict"].value_counts().reset_index()
df_cleaned = df[~df['BCR_CTstrict'].str.contains('NA', na=False)]
df_cleaned["rank"] = range(0, len(df_cleaned))
df_cleaned["BCR_Trace_Clonotype"]  = "BCR_Trace_Clonotype" + "_" + df_cleaned["rank"].astype(str)
df_merge = pd.merge(adata.obs,df_cleaned,left_on="BCR_CTstrict",right_on="BCR_CTstrict",how="left")
print(list(adata.obs.index)==list(df_merge["BC"]))
adata.obs["BCR_Trace_Clonotype"] = list(df_merge["BCR_Trace_Clonotype"])
adata.obs.to_csv("/path/UCART_SLE_6Samples_Bcell_TracingBCRClone.csv")